In [11]:
import os
import sys
import cv2
import glob
import imutils
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from skimage.feature import hog

In [2]:
all_dir = []
x = 11
ppc = 16
for _, dirnames, filenames in os.walk('image'):
    all_dir.append(dirnames)
image = []
classs = []
for i in (all_dir[0]):
     for _, dirnames, filenames in os.walk('image/'+i):
        for j in filenames:
            img = cv2.imread('image/'+i+'/'+j)
            img = cv2.resize(img, (180, 180))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            laplacian = cv2.Laplacian(gray,cv2.CV_64F)
            sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=x) #13
            sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=x)
            sobelxy = cv2.Sobel(gray,cv2.CV_64F,1,1,ksize=x)
            equ = cv2.equalizeHist(gray)
            fd,hog_image = hog(equ, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
            
            ridge_filter = cv2.ximgproc.RidgeDetectionFilter_create()
            gra = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            ridges = ridge_filter.getRidgeFilteredImage(equ)
            #ridges = ridge_filter.getRidgeFilteredImage(img)
            
            a = np.array(fd)
            a = a.flatten()
            b = np.array(img)
            b=b.flatten()
            image.append(a)
            classs.append(i)
np.array(image).shape

/usr/local/lib/python3.6/site-packages/skimage/feature/_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


(112, 8192)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(image, classs, test_size=0.20, random_state=42)

In [7]:
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train, y_train)

print(clf.best_estimator_)

SVC(C=1000.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [16]:
y_pred = clf.predict(image)
print(accuracy_score(classs, y_pred))

1.0


In [8]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

     Alamin       1.00      1.00      1.00         7
      Faruq       1.00      1.00      1.00         8
      Monir       1.00      1.00      1.00         3
     jaydeb       1.00      1.00      1.00         5

avg / total       1.00      1.00      1.00        23



In [15]:
print(confusion_matrix(y_test, y_pred))

             precision    recall  f1-score   support

     Alamin       1.00      1.00      1.00         7
      Faruq       1.00      1.00      1.00         8
      Monir       1.00      1.00      1.00         3
     jaydeb       1.00      1.00      1.00         5

avg / total       1.00      1.00      1.00        23

[[7 0 0 0]
 [0 8 0 0]
 [0 0 3 0]
 [0 0 0 5]]
